In [1]:
from gw_signal_tools.types.ndseries import NDSeries, NDFrequencySeries, NDTimeSeries
# from gw_signal_tools.types.network_waveform import NDWaveform
from gw_signal_tools.types.network_waveform import NetworkWaveform
from gw_signal_tools.types.series_matrix_with_units import SeriesMatrixWithUnits

import numpy as np
import astropy.units as u
from gwpy.types import Series
from gwpy.frequencyseries import FrequencySeries
from gwpy.timeseries import TimeSeries

/home/user/miniconda3/envs/gw_signal_tools_light/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


## Notes

In [2]:
y_vals = np.array([1, 2, 3])
x_vals = np.array([0, 1, 2])
test = Series(value=y_vals, xindex=x_vals, unit=u.s)

test_matrix = np.array([test, 2*test])

print(test_matrix)
print(test_matrix.T)
print(test_matrix @ test_matrix.T)  # Works, but returns numeric values
print(y_vals @ y_vals.T)
print(test*test)

# print(SeriesMatrixWithUnits(test_matrix, u.dimensionless_unscaled))

[[1 2 3]
 [2 4 6]]
[[1 2]
 [2 4]
 [3 6]]
[[14 28]
 [28 56]]
14
Series([1, 4, 9]
       unit: s2,
       name: None,
       epoch: None,
       channel: None,
       x0: 0.0,
       dx: 1.0,
       xindex: [0. 1. 2.])


Problem: np.ndenumerate goes through every value. For np.ndindex, we
can make control the indices since shape is passed. If we pass
self.shape, then controlling this attribute appropriately for
SerierMatrixWithUnits should yield desired behaviour, that each
element is Series then

Idea: make a single index? Because numpy arrays throw it away, no?
And then control that (i) every element has same one in initialization
and (ii) that when setting new element, this has same as all others
(which could just be handled via one that is stored in attribute)

Uhhh, could be nice because of following: then no need to inherit
from MatrixWithUnits, unit property would be kind of useless anyway
(maybe not; but certain functions like plot also don't make sense).
Instead, we make xindex property in the described manner and then
have as values the Series values only (ah, so I guess unit needed
too). But then we can really use numpy array multiplication that
also works in much more complicated shapes, right? And we add as
additional test for SeriesMatrixWithUnits that the respective xindex
are compatible
-> since we do need unit I think (for value=Series.value to work). So
   maybe make BaseMatrix class where all operations are defined and
   then we have MatrixWithUnits as instance that holds numeric values
   (with plotting and all of this stuff), while SeriesMatrixWithUnits
   is more basic and only allows operations with suitable types


Perhaps even more convenient: just subclass Series? Alternative would
be to add self.xindex = property(Series.xindex) (no idea if this is
correct syntax, but that's not the point) to new subclass of
MatrixWithUnits

## NDSeries Tests

In [3]:
nd_val = np.array([[1, 2], [3, 4]])
nd_series = NDSeries(nd_val)
print(nd_series)

NDSeries([[1, 2],
          [3, 4]]
         unit: dimensionless,
         name: None,
         epoch: None,
         channel: None,
         x0: 0.0,
         dx: 1.0,
         xindex: [0. 1.])


Test item getting

In [4]:
print(nd_series.value.ndim, nd_series[0].value.ndim)

print(nd_series[0])

2 1
Series([1, 2]
       unit: dimensionless,
       name: None,
       epoch: None,
       channel: None,
       x0: 0.0,
       dx: 1.0,
       xindex: [0. 1.])


In [5]:
print(Series(nd_series[0]))

Series([1, 2]
       unit: dimensionless,
       name: None,
       epoch: None,
       channel: None,
       x0: 0.0,
       dx: 1.0,
       xindex: [0. 1.])


Test slicing

In [6]:
print(nd_series[0,0])

1.0


In [7]:
print(nd_series[0, :])
print(nd_series[:, 0])
print(nd_series[:, 0:1])
# print(nd_series.xindex[0:1])
print(nd_series.value_at(0))

NDSeries([1, 2]
         unit: dimensionless,
         name: None,
         epoch: None,
         channel: None,
         x0: 0.0,
         dx: 1.0,
         xindex: [0. 1.])
[1. 3.]
[[1.]
 [3.]]
[1. 3.]


In [8]:
print(nd_series.value[:, 0])
print(nd_val[:, 0])

[1 3]
[1 3]


In [9]:
nd_val[:, 0]

array([1, 3])

Testing of slicing and index getting, seems weird (was at beginning)

In [10]:
from gw_signal_tools.types import MatrixWithUnits

In [11]:
example_values = np.array([[42, 24], [18, 96]])
example_units = np.array([[u.s, u.m], [u.m, u.s]])
example_matrix = MatrixWithUnits(example_values, example_units)

In [12]:
print(example_matrix[:, 0])
print(example_matrix[:, 0:1])

array([<Quantity 42. s>, <Quantity 18. m>], dtype=object)
array([[<Quantity 42. s>],
       [<Quantity 18. m>]], dtype=object)


Now test inherited types

In [13]:
freq_series = NDFrequencySeries(nd_val)
print(freq_series)
print(freq_series.f0)  # Test that it works

NDFrequencySeries([[1, 2],
                   [3, 4]]
                  unit: dimensionless,
                  f0: 0.0 Hz,
                  df: 1.0 Hz,
                  epoch: None,
                  name: None,
                  channel: None)
0.0 Hz


In [14]:
time_series = NDTimeSeries(nd_val)
print(time_series)

NDTimeSeries([[1, 2],
              [3, 4]]
             unit: dimensionless,
             t0: 0.0 s,
             dt: 1.0 s,
             name: None,
             channel: None)


## NetworkWaveform Tests

In [15]:
print(Series._metadata_slots)
print(FrequencySeries._metadata_slots)
print(TimeSeries._metadata_slots)

('name', 'epoch', 'channel', 'x0', 'dx', 'xindex')
('name', 'epoch', 'channel', 'x0', 'dx', 'xindex')
('name', 'epoch', 'channel', 'x0', 'dx', 'xindex')


In [16]:
np.equal(np.zeros((2, 2)), np.ones((2, 2)))

array([[False, False],
       [False, False]])

In [17]:
NDSeries.from_series_list([nd_series[0], nd_series[1]])

<NDSeries([[1, 2],
           [3, 4]]
          unit=Unit(dimensionless),
          name=None,
          epoch=None,
          channel=None,
          x0=<Quantity 0.>,
          dx=<Quantity 1.>,
          xindex=<Index [0., 1.]>)>

In [18]:
# test_wf = NetworkWaveform(None, nd_series)  # Does not work
test_wf = NetworkWaveform.from_ndseries(nd_series, [None, None])
print(test_wf)

NetworkWaveform([[1, 2],
                 [3, 4]]
                unit: dimensionless,
                name: None,
                epoch: None,
                channel: None,
                x0: 0.0,
                dx: 1.0,
                xindex: [0. 1.])


## SeriesMatrix Tests

In [19]:
test_series = Series([2, 3, 4, 5])

test_matrix = SeriesMatrixWithUnits(
    [test_series, test_series],
    xindex=test_series.xindex
)

print(test_matrix)
print(test_matrix.xindex)

array([[<Quantity 2.>, <Quantity 3.>, <Quantity 4.>,
        <Quantity 5.>],
       [<Quantity 2.>, <Quantity 3.>, <Quantity 4.>,
        <Quantity 5.>]], dtype=object)
[0. 1. 2. 3.]


In [20]:
test_matrix_T = SeriesMatrixWithUnits(np.transpose([test_series, test_series]), xindex=test_series.xindex)
print(test_matrix_T @ test_matrix)
print(test_matrix @ test_matrix_T)

array([[<Quantity 8.>, <Quantity 12.>, <Quantity 16.>,
        <Quantity 20.>],
       [<Quantity 12.>, <Quantity 18.>, <Quantity 24.>,
        <Quantity 30.>],
       [<Quantity 16.>, <Quantity 24.>, <Quantity 32.>,
        <Quantity 40.>],
       [<Quantity 20.>, <Quantity 30.>, <Quantity 40.>,
        <Quantity 50.>]], dtype=object)
array([[<Quantity 54.>, <Quantity 54.>],
       [<Quantity 54.>, <Quantity 54.>]], dtype=object)


In [21]:
print(test_matrix.value)
print(test_matrix_T.value)
# print(np.transpose(test_matrix.value))

[[2 3 4 5]
 [2 3 4 5]]
[[2 2]
 [3 3]
 [4 4]
 [5 5]]


In [22]:
full_matrix = SeriesMatrixWithUnits(
    [[test_series, test_series], [test_series, test_series]],
    xindex=test_series.xindex
)

In [23]:
print(full_matrix @ test_matrix_T)

array([[[<Quantity 54.>, <Quantity 54.>],
        [<Quantity 54.>, <Quantity 54.>]],

       [[<Quantity 54.>, <Quantity 54.>],
        [<Quantity 54.>, <Quantity 54.>]]], dtype=object)


In [25]:
arr = np.array([test_series, test_series])
print(arr)
print(np.transpose(arr, axes=(0, 1)))
print(np.transpose(arr, axes=(1, 0)))
# print(np.transpose(arr, axes=(2, 0)))

[[2 3 4 5]
 [2 3 4 5]]
[[2 3 4 5]
 [2 3 4 5]]
[[2 2]
 [3 3]
 [4 4]
 [5 5]]


In [26]:
arr_T = np.atleast_2d(test_series.value).T
print(arr_T)

[[2]
 [3]
 [4]
 [5]]


In [27]:
arr = test_series.value
print(arr @ arr_T)
# print(arr_T @ arr)

arr_arr = np.array([arr, arr])
arr_arr_T = np.array([arr_T, arr_T])
print(arr_arr @ arr_arr_T)
# print(arr_arr_T @ arr_arr)

print(np.tensordot(arr, arr, axes=(0, 0)))
print(np.tensordot(arr_arr, arr_arr, axes=(0, 0)))
print(np.tensordot(arr_arr, arr_arr, axes=(1, 1)))

[54]
[[[54]
  [54]]

 [[54]
  [54]]]
54
[[ 8 12 16 20]
 [12 18 24 30]
 [16 24 32 40]
 [20 30 40 50]]
[[54 54]
 [54 54]]


In [28]:
print(arr @ arr_arr_T)
print(arr_arr @ arr_T)
# print(arr_T @ arr_arr @ arr_T)

[[54]
 [54]]
[[54]
 [54]]


In [87]:
# arr_arr_arr_arr = np.array([[arr, arr], [arr, arr]])
arr_arr_arr_arr = np.concatenate([arr, arr, arr, arr]).reshape((2, 2, 4))  # Equivalent
print(arr_arr_arr_arr)
print(arr_arr_arr_arr[:, :, 0])
print(arr_arr_arr_arr[0, 0, :])

[[[2 3 4 5]
  [2 3 4 5]]

 [[2 3 4 5]
  [2 3 4 5]]]
[[2 2]
 [2 2]]
[2 3 4 5]


In [69]:
print(arr_arr)
print(arr_arr[0, :])
print(arr_arr_T)

[[2 3 4 5]
 [2 3 4 5]]
[2 3 4 5]
[[[2]
  [3]
  [4]
  [5]]

 [[2]
  [3]
  [4]
  [5]]]


In [82]:
print(np.sum(arr_arr_arr_arr, axis=2))
print(np.sum(arr_arr, axis=1))

[[14 14]
 [14 14]]
[14 14]


In [92]:
print(np.einsum('ijk, lk', arr_arr_arr_arr, arr_arr))
# -- Same as np.tensordot(arr_arr_arr_arr, arr_arr, axes=(2, 1))

[[[54 54]
  [54 54]]

 [[54 54]
  [54 54]]]


In [147]:
print(np.einsum('ji, jkl, ml', arr_arr, arr_arr_arr_arr, arr_arr))  # Not the shape that we want
# print(np.einsum('ji, jkl, il', arr_arr, arr_arr_arr_arr, arr_arr))  # Error
print(np.einsum('ik, ijk, jk', arr_arr, arr_arr_arr_arr, arr_arr))  # Let's gooo, a scalar

# -- How can we get equivalent tensordot output?
# print(np.tensordot(np.tensordot(arr_arr, arr_arr_arr_arr, axes=(1, 2)), arr_arr, axes=((1, 2), (0, 1))))  # Error, thus testing below
print(np.tensordot(arr_arr, arr_arr_arr_arr, axes=(1, 2)).shape, arr_arr.shape)
# print(np.tensordot(np.tensordot(arr_arr, arr_arr_arr_arr, axes=(1, 2)), arr_arr, axes=((1, 2), (1, 2))))
n = len(arr)
big_mat = np.zeros(2*(2*n, ))
# np.fill_diagonal(big_mat, arr_arr_arr_arr.reshape((1, 4*n)))
# # np.fill_diagonal(big_mat, arr_arr_arr_arr.reshape(2*n))
# Shit, with this we miss out on half of the entries because Nebendiagonale is not filled

# Following better
np.fill_diagonal(big_mat[:n, :n], arr_arr_arr_arr[0, 0])
np.fill_diagonal(big_mat[:n, n:], arr_arr_arr_arr[0, 1])
np.fill_diagonal(big_mat[n:, :n], arr_arr_arr_arr[1, 0])
np.fill_diagonal(big_mat[n:, n:], arr_arr_arr_arr[1, 1])
# print(big_mat)

flat_arr = arr_arr.reshape((1, 2*n))
print(flat_arr @ big_mat)
print(flat_arr @ big_mat @ flat_arr.T)  # YEEESSS

# print(np.tensordot(arr_arr.reshape((1, 2*n)), big_mat, axes=(0, 1)))

[[[216 216]
  [216 216]]

 [[324 324]
  [324 324]]

 [[432 432]
  [432 432]]

 [[540 540]
  [540 540]]]
896
(2, 2, 2) (2, 4)
[[ 8. 18. 32. 50.  8. 18. 32. 50.]]
[[896.]]


In [139]:
# big_mat
arr_arr_arr_arr.reshape((1, 4*n))#.shape

array([[2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5]])

In general: np.einsum('ik, ijk, jk', arr1, arr2, arr3) is equivalent to np.tensordot(np.tensordot(arr1, arr2, axes=(1, 2)), arr3, axes=((1, 2), (0, 1)))

In [62]:
print(np.tensordot(arr_arr_arr_arr, arr_arr, axes=(0, 0)))
print(np.tensordot(arr_arr_arr_arr, arr_arr, axes=(1, 0)))
print(np.tensordot(arr_arr_arr_arr, arr_arr, axes=(2, 1)))

[[[ 8 12 16 20]
  [12 18 24 30]
  [16 24 32 40]
  [20 30 40 50]]

 [[ 8 12 16 20]
  [12 18 24 30]
  [16 24 32 40]
  [20 30 40 50]]]
[[[ 8 12 16 20]
  [12 18 24 30]
  [16 24 32 40]
  [20 30 40 50]]

 [[ 8 12 16 20]
  [12 18 24 30]
  [16 24 32 40]
  [20 30 40 50]]]
[[[54 54]
  [54 54]]

 [[54 54]
  [54 54]]]


In [64]:
# print(np.tensordot(arr_arr_arr_arr, arr_arr_T, axes=(0, 0)))
# print(np.tensordot(arr_arr_arr_arr, arr_arr_T, axes=(1, 0)))
# print(np.tensordot(arr_arr_arr_arr, arr_arr_T, axes=(2, 0)))
print(np.tensordot(arr_arr_arr_arr, arr_arr_T, axes=(2, 1)))

[[[[54]
   [54]]

  [[54]
   [54]]]


 [[[54]
   [54]]

  [[54]
   [54]]]]
